In [1]:
import numpy as np
import pandas as pd

In [2]:
trade_df = pd.read_stata("tradeflows.dta")
trade_df.head()

,year,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,...,rta,comleg,comcur,acp_to_eu,gsp,eu_to_acp,gsp_rec,flow,validmirror,family
0,1999,ABW,AGO,0,0,9587.315563,NaN,1722.905029,NaN,AW,...,0,1,0,0,0,0,0,0.0,0,
1,2000,ABW,AGO,0,0,9587.315563,NaN,1858.659180,NaN,AW,...,0,1,0,0,0,0,0,0.0,1,
2,2001,ABW,AGO,0,0,9587.315563,NaN,1898.882690,NaN,AW,...,0,1,0,0,0,0,0,0.0,1,
3,2002,ABW,AGO,0,0,9587.315563,NaN,1911.173218,NaN,AW,...,0,1,0,0,0,0,0,0.0,1,
4,2003,ABW,AGO,0,0,9587.315563,0.099,NaN,NaN,AW,...,0,1,0,0,0,0,0,0.0,1,


In [3]:
trade_df.columns

Index(['year', 'iso_o', 'iso_d', 'contig', 'comlang_off', 'distw', 'pop_o',
       'gdp_o', 'gdpcap_o', 'iso2_o', 'pop_d', 'gdp_d', 'gdpcap_d', 'iso2_d',
       'heg_d', 'conflict', 'indepdate', 'heg_o', 'col_to', 'col_fr',
       'col_hist', 'col_cur', 'sever', 'sib_conflict', 'gatt_o', 'gatt_d',
       'rta', 'comleg', 'comcur', 'acp_to_eu', 'gsp', 'eu_to_acp', 'gsp_rec',
       'flow', 'validmirror', 'family'],
      dtype='object')

In [4]:
polity_scores_df = pd.read_excel("polity.xls")
polity_scores_df.head()

,cyear,ccode,scode,country,year,flag,fragment,democ,autoc,polity,...,interim,bmonth,bday,byear,bprec,post,change,d4,sf,regtrans
0,7001800,700,AFG,Afghanistan,1800,0,NaN,1,7,-6,...,NaN,1.0,1.0,1800.0,1.0,-6.0,88.0,1.0,NaN,NaN
1,7001801,700,AFG,Afghanistan,1801,0,NaN,1,7,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7001802,700,AFG,Afghanistan,1802,0,NaN,1,7,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7001803,700,AFG,Afghanistan,1803,0,NaN,1,7,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7001804,700,AFG,Afghanistan,1804,0,NaN,1,7,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
same_countries = sorted(list(set(trade_df['iso_o'].unique()) & set(polity_scores_df['scode'].unique())))
print(same_countries)
print("Polity - Same =", set(polity_scores_df['scode'].unique()).difference(set(same_countries)))

['AFG', 'AGO', 'ALB', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BIH', 'BLR', 'BOL', 'BRA', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CMR', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CYP', 'CZE', 'DEU', 'DJI', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'GAB', 'GBR', 'GEO', 'GHA', 'GMB', 'GNB', 'GNQ', 'GRC', 'GTM', 'GUY', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KOR', 'KWT', 'LBN', 'LBR', 'LBY', 'LKA', 'LSO', 'LTU', 'LUX', 'LVA', 'MAR', 'MDA', 'MDG', 'MEX', 'MKD', 'MLI', 'MNG', 'MOZ', 'MRT', 'MUS', 'MWI', 'MYS', 'NAM', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PNG', 'POL', 'PRT', 'PRY', 'QAT', 'RUS', 'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLB', 'SLE', 'SLV', 'SOM', 'SUR', 'SVN', 'SWE', 'SWZ', 'SYR', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TTO', 'TUN', 'TUR', 'TZA', 'UGA', 'UKR', 'URY'

In [6]:
polity_scores_df = polity_scores_df.loc[:,['scode', 'country', 'year', 'polity']]
polity_scores_df.head()

,scode,country,year,polity
0,AFG,Afghanistan,1800,-6
1,AFG,Afghanistan,1801,-6
2,AFG,Afghanistan,1802,-6
3,AFG,Afghanistan,1803,-6
4,AFG,Afghanistan,1804,-6


In [7]:
# Remove all rows prior to 1950
trade_df = trade_df[trade_df['year'] >= 1950]
polity_scores_df = polity_scores_df[polity_scores_df['year'] >= 1950]
trade_df.shape

(1181428, 36)

In [8]:
# Remove all rows with iso_o not in same countries
trade_df = trade_df[trade_df['iso_o'].isin(same_countries)]
# Remove all rows with iso_d not in same countries
trade_df = trade_df[trade_df['iso_d'].isin(same_countries)]
trade_df.reset_index(drop=True, inplace=True)

In [9]:
# Create new column with trade link (iso_d + iso_d)
trade_df["tradelink"] = trade_df.apply(lambda row: row['iso_o'] + '-' + row['iso_d'], axis=1)
trade_df.reset_index(drop=True, inplace=True)

In [10]:
trade_df.describe()

,year,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,pop_d,gdp_d,gdpcap_d,...,gatt_d,rta,comleg,comcur,acp_to_eu,gsp,eu_to_acp,gsp_rec,flow,validmirror
count,746640.000000,746640.000000,746640.000000,746640.000000,742286.000000,6.920980e+05,691741.000000,743347.000000,7.098420e+05,709513.000000,...,746640.000000,746640.000000,746640.000000,746640.000000,746640.000000,746640.000000,746640.000000,746640.000000,746640.000000,746640.000000
mean,1982.277314,0.025188,0.172039,7461.426431,36.864864,1.490802e+05,4221.940918,36.496063,1.457358e+05,4156.484863,...,0.646013,0.032293,0.377067,0.017415,0.025166,0.082044,0.024949,0.082244,180.937075,0.843058
std,16.246874,0.156694,0.377414,4357.002232,121.738029,6.813041e+05,7855.582031,119.284630,6.725719e+05,7788.756836,...,0.478206,0.176776,0.484652,0.130813,0.156629,0.274431,0.155970,0.274737,2421.789324,0.363746
min,1950.000000,0.000000,0.000000,114.637288,0.025000,2.520352e+01,6.813283,0.027000,2.794697e+01,6.813283,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1969.000000,0.000000,0.000000,3987.383312,3.111720,2.093200e+03,319.730682,3.495100,2.103000e+03,312.037567,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1984.000000,0.000000,0.000000,6945.261626,8.233300,8.590567e+03,970.799744,8.527777,8.382545e+03,956.226257,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.235597,1.000000
75%,1997.000000,0.000000,0.000000,10431.940200,24.593000,5.794614e+04,3546.669189,25.271799,5.512262e+04,3461.856934,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.460000,1.000000
max,2006.000000,1.000000,1.000000,19650.127278,1311.797729,1.320182e+07,66964.367188,1311.797729,1.320182e+07,66964.367188,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,348420.600000,1.000000


In [11]:
trade_df = pd.merge(left=trade_df, right=polity_scores_df, left_on=['year','iso_o'], right_on=['year','scode']).drop(columns=['country', 'scode'])
trade_df = trade_df.rename(columns={'polity':'polity_o'})
trade_df = pd.merge(left=trade_df, right=polity_scores_df, left_on=['year','iso_d'], right_on=['year','scode']).drop(columns=['country', 'scode'])
trade_df = trade_df.rename(columns={'polity':'polity_d'})
trade_df.head()

,year,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,...,acp_to_eu,gsp,eu_to_acp,gsp_rec,flow,validmirror,family,tradelink,polity_o,polity_d
0,1952,AFG,ARG,0,0,15341.161651,8.425,NaN,NaN,AF,...,0,0,0,0,0.0,0,,AFG-ARG,-10,-9
1,1952,AUS,ARG,0,0,12044.573996,8.691,8007.00000,921.297913,AU,...,0,0,0,0,0.0,1,,AUS-ARG,10,-9
2,1952,AUT,ARG,0,0,11751.146582,6.928,NaN,NaN,AT,...,0,0,0,0,23.0,1,,AUT-ARG,10,-9
3,1952,BEL,ARG,0,0,11305.285880,8.730,8132.00000,931.500610,BE,...,0,0,0,0,34.0,1,,BEL-ARG,10,-9
4,1952,BGR,ARG,0,0,12115.572212,7.275,1267.23999,174.191055,BG,...,0,0,0,0,0.0,0,,BGR-ARG,-7,-9


In [12]:
# Append difference in scores column to trade_df
trade_df['polity_dist'] = abs(trade_df['polity_o'] - trade_df['polity_d'])
trade_df.head(20)

,year,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,...,gsp,eu_to_acp,gsp_rec,flow,validmirror,family,tradelink,polity_o,polity_d,polity_dist
0,1952,AFG,ARG,0,0,15341.161651,8.425000,NaN,NaN,AF,...,0,0,0,0.0,0,,AFG-ARG,-10,-9,1
1,1952,AUS,ARG,0,0,12044.573996,8.691000,8007.000000,921.297913,AU,...,0,0,0,0.0,1,,AUS-ARG,10,-9,19
2,1952,AUT,ARG,0,0,11751.146582,6.928000,NaN,NaN,AT,...,0,0,0,23.0,1,,AUT-ARG,10,-9,19
3,1952,BEL,ARG,0,0,11305.285880,8.730000,8132.000000,931.500610,BE,...,0,0,0,34.0,1,,BEL-ARG,10,-9,19
4,1952,BGR,ARG,0,0,12115.572212,7.275000,1267.239990,174.191055,BG,...,0,0,0,0.0,0,,BGR-ARG,-7,-9,2
5,1952,BOL,ARG,1,1,1865.839405,2.883000,303.850006,105.393692,BO,...,0,0,0,1.9,1,ESP,BOL-ARG,-8,-9,1
6,1952,BRA,ARG,1,0,2391.846403,56.603001,6211.000000,109.729164,BR,...,0,0,0,114.0,1,,BRA-ARG,5,-9,14
7,1952,CAN,ARG,0,0,9391.460934,14.786000,25117.000000,1698.701416,CA,...,0,0,0,12.4,1,,CAN-ARG,10,-9,19
8,1952,CHE,ARG,0,0,11232.932834,4.815000,5370.000000,1115.264771,CH,...,0,0,0,23.1,1,,CHE-ARG,10,-9,19
9,1952,CHL,ARG,1,1,1156.725891,6.378000,2360.629883,370.120728,CL,...,0,0,0,45.4,1,ESP,CHL-ARG,2,-9,11


In [13]:
import math

In [14]:
trade_df['log_gdp_o'] = np.log(trade_df.gdp_o)
trade_df['log_gdp_d'] = np.log(trade_df.gdp_d)
trade_df['log_flow'] = np.log(trade_df.flow)
trade_df['log_flow'] = trade_df['log_flow'].apply(lambda x: np.log(1+x))
trade_df['log_distw'] = trade_df['distw'].apply(lambda x: np.log(1+x))
trade_df['log_polity_dist'] = trade_df['polity_dist'].apply(lambda x: np.log(int(1+x)))
trade_df['log_col_hist'] = trade_df['col_hist'].apply(lambda x: np.log(int(1+x)))
trade_df.head(10)

/home/topher-lo/anaconda3/envs/supas/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


,year,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,...,tradelink,polity_o,polity_d,polity_dist,log_gdp_o,log_gdp_d,log_flow,log_distw,log_polity_dist,log_col_hist
0,1952,AFG,ARG,0,0,15341.161651,8.425000,NaN,NaN,AF,...,AFG-ARG,-10,-9,1,NaN,8.942325,NaN,9.638360,0.693147,0.0
1,1952,AUS,ARG,0,0,12044.573996,8.691000,8007.000000,921.297913,AU,...,AUS-ARG,10,-9,19,8.988071,8.942325,NaN,9.396453,2.995732,0.0
2,1952,AUT,ARG,0,0,11751.146582,6.928000,NaN,NaN,AT,...,AUT-ARG,10,-9,19,NaN,8.942325,1.419607,9.371791,2.995732,0.0
3,1952,BEL,ARG,0,0,11305.285880,8.730000,8132.000000,931.500610,BE,...,BEL-ARG,10,-9,19,9.003562,8.942325,1.509918,9.333114,2.995732,0.0
4,1952,BGR,ARG,0,0,12115.572212,7.275000,1267.239990,174.191055,BG,...,BGR-ARG,-7,-9,2,7.144597,8.942325,NaN,9.402329,1.098612,0.0
5,1952,BOL,ARG,1,1,1865.839405,2.883000,303.850006,105.393692,BO,...,BOL-ARG,-8,-9,1,5.716534,8.942325,0.495826,7.532002,0.693147,0.0
6,1952,BRA,ARG,1,0,2391.846403,56.603001,6211.000000,109.729164,BR,...,BRA-ARG,5,-9,14,8.734077,8.942325,1.746797,7.780239,2.708050,0.0
7,1952,CAN,ARG,0,0,9391.460934,14.786000,25117.000000,1698.701416,CA,...,CAN-ARG,10,-9,19,10.131300,8.942325,1.257806,9.147663,2.995732,0.0
8,1952,CHE,ARG,0,0,11232.932834,4.815000,5370.000000,1115.264771,CH,...,CHE-ARG,10,-9,19,8.588583,8.942325,1.420655,9.326694,2.995732,0.0
9,1952,CHL,ARG,1,1,1156.725891,6.378000,2360.629883,370.120728,CL,...,CHL-ARG,2,-9,11,7.766684,8.942325,1.571842,7.054213,2.484907,0.0


In [15]:
trade_df = trade_df.set_index(['tradelink', 'year'])

In [16]:
trade_df.to_csv("trade_polity.csv")